In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np

def generate_biased_dataset(size):
    data = []
    while len(data) < size:
        a = random.randint(0, 2048)
        b = random.randint(0, 2048)
        if a % 5 != 0 or b % 5 != 0:
            continue
        y = a ^ b
        data.append(((a / 2048.0, b / 2048.0), y / 2048.0))
    return data

train_data = generate_biased_dataset(10000)
test_data = generate_biased_dataset(1000)

class XORDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor([y], dtype=torch.float32)

train_loader = torch.utils.data.DataLoader(XORDataset(train_data), batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(XORDataset(test_data), batch_size=64, shuffle=False)

class XORNet(nn.Module):
    def __init__(self):
        super(XORNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = XORNet()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(x_batch).squeeze()
            loss = criterion(outputs, y_batch.squeeze())
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            outputs = model(x_batch).squeeze()
            predictions = (outputs > 0.5).float()
            correct += (predictions == y_batch.squeeze()).sum().item()
            total += y_batch.size(0)
    print(f"Test Accuracy: {correct / total * 100:.2f}%")

def evaluate_model_on_varied_numbers(model):
    model.eval()
    results = []
    with torch.no_grad():
        for a in range(0, 2050, 50):
            for b in range(0, 256, 10):
                a_norm = a / 2048.0
                b_norm = b / 2048.0
                x = torch.tensor([[a_norm, b_norm]], dtype=torch.float32)
                output = model(x).item()
                predicted = round(output * 2048)
                expected = a ^ b

                results.append({
                    "a": a,
                    "b": b,
                    "expected": expected,
                    "predicted": predicted,
                    "error": abs(expected - predicted)
                })

    results.sort(key=lambda x: x["error"], reverse=True)

    print("Top 10 worst predictions:")
    for r in results[:10]:
        print(f"a={r['a']}, b={r['b']}, expected={r['expected']}, predicted={r['predicted']}, error={r['error']}")

train_model(model, train_loader, criterion, optimizer, epochs=10)
test_model(model, test_loader)
print("")
evaluate_model_on_varied_numbers(model)

Epoch 1/10, Loss: 0.0463
Epoch 2/10, Loss: 0.0285
Epoch 3/10, Loss: 0.0264
Epoch 4/10, Loss: 0.0254
Epoch 5/10, Loss: 0.0241
Epoch 6/10, Loss: 0.0236
Epoch 7/10, Loss: 0.0230
Epoch 8/10, Loss: 0.0222
Epoch 9/10, Loss: 0.0212
Epoch 10/10, Loss: 0.0200
Test Accuracy: 0.20%

Top 10 worst predictions:
a=1050, b=230, expected=1276, predicted=975, error=301
a=1800, b=160, expected=1960, predicted=1668, error=292
a=1800, b=180, expected=1980, predicted=1692, error=288
a=1800, b=150, expected=1950, predicted=1664, error=286
a=1850, b=130, expected=1976, predicted=1690, error=286
a=1550, b=240, expected=1790, predicted=1507, error=283
a=1050, b=160, expected=1210, predicted=929, error=281
a=1050, b=200, expected=1234, predicted=957, error=277
a=1800, b=170, expected=1954, predicted=1677, error=277
a=1050, b=240, expected=1258, predicted=982, error=276


Top 10 numbers with the most factors and sub-factors divisible by 5:
Number: 1800, Factors and sub-factors divisible by 5: 24
Number: 1200, Factors and sub-factors divisible by 5: 20
Number: 1680, Factors and sub-factors divisible by 5: 20
Number: 900, Factors and sub-factors divisible by 5: 18
Number: 1260, Factors and sub-factors divisible by 5: 18
Number: 1440, Factors and sub-factors divisible by 5: 18
Number: 1500, Factors and sub-factors divisible by 5: 18
Number: 1980, Factors and sub-factors divisible by 5: 18
Number: 600, Factors and sub-factors divisible by 5: 16
Number: 840, Factors and sub-factors divisible by 5: 16
